# AstraDB / Cassandra Query Verification

This notebook demonstrates how to execute the mission-critical business queries against the data model designed for AstraDB.

The schema has been optimized for these specific access patterns:
1. Daily Cost & Requests (Range Query)
2. Top-N Services
3. Support Tickets Evolution
4. Monthly Revenue
5. GenAI Usage


In [ ]:
# Query parameters
ORG_ID = "org_123_test"
START_DATE = "2023-01-01"
END_DATE = "2023-01-31"


### 1. Daily Cost & Requests by Org and Service (Range Query)
**Requirement**: Daily costs and requests by organization and service within a date range.

**Optimization**: We moved `date` to Clustering Key to enable range slices (`>=`, `<=`).

In [ ]:
query_1 = """
SELECT date, service, total_cost_usd, total_requests
FROM org_daily_usage_by_service
WHERE org_id = ? 
  AND date >= ? 
  AND date <= ?
"""

print("Query 1 (Range):", query_1)

### 2. Top-N Services by Cost (Last 14 Days)
**Requirement**: Top-N services by accumulated cost in the last 14 days for an organization.

**Strategy**: Application-side aggregation. Cassandra is efficient at retrieving the range; the application aggregates.


In [ ]:
# Step 1: Fetch data for the last 14 days
query_2 = """
SELECT service, total_cost_usd
FROM org_daily_usage_by_service
WHERE org_id = ?
  AND date >= ?  -- (Today - 14 days)
"""

# Step 2: In Python (Application Layer)
# df = execute(query_2, [org_id, start_date])
# top_n = df.groupby('service')['total_cost_usd'].sum().nlargest(N)
print("Query 2 (Fetch):", query_2)

### 3. Tickets Evolution & SLA Breach Rate (Last 30 Days)
**Requirement**: Evolution of critical tickets and SLA breach rate per day (last 30 days).

In [ ]:
query_3 = """
SELECT date, ticket_count, critical_tickets_count, sla_breach_rate, sla_breach_count
FROM tickets_by_org_date
WHERE org_id = ?
  AND date >= ? -- (Today - 30 days)
"""
print("Query 3:", query_3)

### 4. Monthly Revenue (Financial Truth)
**Requirement**: Monthly revenue with credits/taxes applied from Billing System.
**Source**: Now sourced from `billing_processed` Silver table.

In [ ]:
query_4 = """
SELECT 
    year, 
    month, 
    subtotal_usd,
    credits_usd,
    taxes_usd,
    total_revenue_usd,
    exchange_rate_used,
    active_days
FROM revenue_by_org_month
WHERE org_id = ?
ORDER BY year DESC, month DESC
LIMIT 12
"""
print("Query 4:", query_4)

### 5. GenAI Tokens & Cost (Daily)
**Requirement**: GenAI tokens and actual cost per day (aggregated across all GenAI services).
**Optimization**: Optimized PK allows efficient retrieval per Org for a timeline.

In [ ]:
query_5 = """
SELECT date, total_tokens, genai_cost_usd
FROM genai_tokens_by_org_date
WHERE org_id = ?
  AND date >= ?
"""
print("Query 5:", query_5)